In [1]:
import pandas as pd

In [2]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file("./EDA/data/key.json")

In [3]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id
)

In [4]:
safe = '''
SELECT 
  T1.county_name,
  T1.town_name,
  T1.area,
  T1.pop_total,
  T1.pop_men,
  T1.pop_women,
  T1.men_2030,
  T1.women_2030,
  T2.cctv_num,
  COALESCE(T3.light_num, 0) AS light_num,
  COALESCE(T4.house_num, 0) AS house_num,
  COALESCE(T5.police_num, 0) AS police_num
FROM 
  (
    SELECT 
      pop.county_name,
      pop.town_name,
      sum(area.Subtotal) as area,
      SUM(pop.total) AS pop_total,
      sum(pop.men) as pop_men,
      sum(pop.women) as pop_women,
      SUM(_20_29_men) + SUM(_30_39_men) AS men_2030,
      SUM(_20_29_women) + SUM(_30_39_women) AS women_2030
    FROM 
      `my-fantazip.fantazip.population_by_age` AS pop
    inner join
      `my-fantazip.fantazip.town_area_final` AS area
    on
      pop.county_name = area.county_name
      and pop.town_name = area.town_name
    GROUP BY
      pop.county_name,
      pop.town_name
  ) AS T1
INNER JOIN
  (
    SELECT 
      county_name,
      town_name,
      SUM(install_num) AS cctv_num
    FROM 
      `my-fantazip.fantazip.cctv`
    GROUP BY
      county_name,
      town_name
  ) AS T2
ON
  T1.county_name = T2.county_name
  AND T1.town_name = T2.town_name
LEFT OUTER JOIN
  (
    SELECT 
      county_name,
      town_name,
      SUM(install_num) AS light_num
    FROM 
      `my-fantazip.fantazip.light`
    GROUP BY
      county_name,
      town_name
  ) AS T3
ON
  T1.county_name = T3.county_name
  AND T1.town_name = T3.town_name
LEFT OUTER JOIN
  (
    SELECT 
      county_name,
      town_name,
      COUNT(town_name) AS house_num
    FROM 
      `my-fantazip.fantazip.safety_house`
    GROUP BY
      county_name,
      town_name
  ) AS T4
ON
  T1.county_name = T4.county_name
  AND T1.town_name = T4.town_name
LEFT OUTER JOIN
  (
    SELECT 
      county_name,
      town_name,
      COUNT(town_name) AS police_num
    FROM 
      `my-fantazip.fantazip.police`
    GROUP BY
      county_name,
      town_name
  ) AS T5
ON
  T1.county_name = T5.county_name
  AND T1.town_name = T5.town_name
'''

In [5]:
safe = client.query(safe).to_dataframe()
safe

,county_name,town_name,area,pop_total,pop_men,pop_women,men_2030,women_2030,cctv_num,light_num,house_num,police_num
0,종로구,신교동,99144.0,1341,600,741,157,177,19,58,1,0
1,종로구,누상동,261803.6,3177,1429,1748,350,422,22,122,0,0
2,종로구,옥인동,414842.7,1854,842,1012,260,308,40,188,0,1
3,종로구,필운동,86207.7,981,448,533,122,146,11,66,2,0
4,종로구,사직동,284141.8,3166,1393,1773,325,418,24,140,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
440,서초구,잠원동,3035875.4,50621,24212,26409,5567,6182,255,590,2,0
441,강남구,삼성동,3193589.8,42657,20583,22074,5531,6182,697,1268,10,2
442,강남구,압구정동,2685131.5,30190,14013,16177,2739,3349,129,44,0,1
443,송파구,신천동,4346310.8,37702,18034,19668,3663,4328,101,102,0,1


In [7]:
safe.to_csv("./concat_by_dong.csv", index=False)

In [61]:
crime = pd.read_csv("./data_final/서울시_범죄현황.csv")
crime

,city_name,county_name,total_crime,total_arrest,murder_crime,murder_arrest,robbery_crime,robbery_arrest,sex_crime,sex_arrest,theft_crime,theft_arrest,violence_crime,violence_arrest
0,서울,종로구,3138,3284,2,3,8,8,228,988,1313,932,1587,1353
1,서울,중구,3071,2119,1,2,5,4,194,123,1365,724,1506,1266
2,서울,용산구,2967,2093,5,3,7,7,280,238,978,508,1697,1337
3,서울,성동구,2194,1597,2,2,2,2,125,97,979,538,1086,958
4,서울,광진구,3619,2498,6,5,-,-,230,174,1670,831,1713,1488
5,서울,동대문구,3253,2231,4,4,2,2,125,106,1556,823,1566,1296
6,서울,중랑구,3599,2671,2,2,2,2,150,121,1508,882,1937,1664
7,서울,성북구,2749,2125,3,3,1,1,158,129,1143,715,1444,1277
8,서울,강북구,2832,2343,5,4,5,6,177,147,872,589,1773,1597
9,서울,도봉구,2141,1569,3,3,-,-,70,54,969,546,1099,966


In [66]:
# '-' 값을 0으로 변환
crime['robbery_crime'] = crime['robbery_crime'].replace('-', '0')
crime['robbery_arrest'] = crime['robbery_arrest'].replace('-', '0')

In [70]:
crime = crime[['county_name', 'total_crime', "murder_crime", "robbery_crime", "sex_crime", "theft_crime", "violence_crime"]]
crime

,county_name,total_crime,murder_crime,robbery_crime,sex_crime,theft_crime,violence_crime
0,종로구,3138,2,8,228,1313,1587
1,중구,3071,1,5,194,1365,1506
2,용산구,2967,5,7,280,978,1697
3,성동구,2194,2,2,125,979,1086
4,광진구,3619,6,0,230,1670,1713
5,동대문구,3253,4,2,125,1556,1566
6,중랑구,3599,2,2,150,1508,1937
7,성북구,2749,3,1,158,1143,1444
8,강북구,2832,5,5,177,872,1773
9,도봉구,2141,3,0,70,969,1099


In [72]:
safe.columns

Index(['county_name', 'town_name', 'area', 'pop_total', 'pop_men', 'pop_women',
       'men_2030', 'women_2030', 'cctv_num', 'light_num', 'house_num',
       'police_num'],
      dtype='object')

In [76]:
safe_gu = safe.groupby("county_name")[['area', 'pop_total', 'pop_men', 'pop_women',
       'men_2030', 'women_2030', 'cctv_num', 'light_num', 'house_num',
       'police_num']].sum()
safe_gu

,area,pop_total,pop_men,pop_women,men_2030,women_2030,cctv_num,light_num,house_num,police_num
county_name,,,,,,,,,,
강남구,39497597.1,549154,262618,286536,68860,77955,6814,10210,112,14
강동구,24590382.7,459389,224450,234939,60982,63912,2541,7503,43,9
강북구,23600408.1,287490,139141,148349,36477,36353,2757,9776,25,9
강서구,33942883.4,562528,269607,292921,83758,91473,2431,8635,36,11
관악구,29568368.9,481872,242598,239274,101562,94240,4624,12401,53,9
광진구,17062820.1,335335,161154,174181,54594,58908,3134,8971,15,10
구로구,20122908.3,392311,192252,200059,55561,56273,4290,8015,31,8
금천구,13020527.6,227457,114471,112986,37784,35149,2400,6269,9,5
노원구,35439122.0,497237,238609,258628,63984,62865,1991,5288,27,8


In [77]:
# 'county_name'을 기준으로 inner join
crime_gu = pd.merge(crime, safe_gu, on='county_name', how='inner')
crime_gu

,county_name,total_crime,murder_crime,robbery_crime,sex_crime,theft_crime,violence_crime,area,pop_total,pop_men,pop_women,men_2030,women_2030,cctv_num,light_num,house_num,police_num
0,종로구,3138,2,8,228,1313,1587,22536925.8,137627,66452,71175,19544,20680,1608,8643,25,18
1,중구,3071,1,5,194,1365,1506,9759468.4,121322,58651,62671,18376,19995,2287,6064,25,14
2,용산구,2967,5,7,280,978,1697,19036534.5,212173,101791,110382,32201,34894,1484,9960,23,7
3,성동구,2194,2,2,125,979,1086,16820149.0,277090,134390,142700,42096,44378,1255,215,15,9
4,광진구,3619,6,0,230,1670,1713,17062820.1,335335,161154,174181,54594,58908,3134,8971,15,10
5,동대문구,3253,4,2,125,1556,1566,14215912.2,340983,167230,173753,52342,53792,2620,10307,39,10
6,중랑구,3599,2,2,150,1508,1937,18497134.6,382284,187392,194892,53345,54355,3748,9330,28,8
7,성북구,2749,3,1,158,1143,1444,24577917.6,424916,203824,221092,58327,62017,3605,9982,34,9
8,강북구,2832,5,5,177,872,1773,23600408.1,287490,139141,148349,36477,36353,2757,9776,25,9
9,도봉구,2141,3,0,70,969,1099,20651010.0,306722,148670,158052,37896,37018,1395,6334,0,7


In [78]:
crime_gu.to_csv("./concat_by_gu.csv", index=False)